In [1]:
import json

In [35]:
with open("resp.json", "r", encoding="utf-8") as f:
    data = json.load(f)

In [37]:
res = {}
models_costs_per_million_tokens = {
    "gpt-4.1-nano": {
        "input": 0.2,
        "output": 0.8,
    },
    "gpt-5-mini": {
        "input": 0.25,
        "output": 2.0,
    },
}
for d in data:
    metrics = d["metrics"]
    for k, v in metrics.items():
        input_tokens = v["prompt_tokens"]
        output_tokens = v["completion_tokens"]

        if k not in res:
            res[k] = {
                "input": input_tokens,
                "output": output_tokens,
                "count": 1,
            }
        else:
            res[k]["count"] += 1
            res[k]["input"] += input_tokens
            res[k]["output"] += output_tokens


model_total_cost = {m: 0 for m in models_costs_per_million_tokens.keys()}
total_posts = max(v["count"] for v in res.values())
for k, v in res.items():
    print(f"\n{k}")

    avg_input_tokens = v["input"] / total_posts
    avg_output_tokens = v["output"] / total_posts
    
    print(f"Average input tokens: {avg_input_tokens:.2f}")
    print(f"Average output tokens: {avg_output_tokens:.2f}")

    print(f"Total cost per post:")

    for m, c in models_costs_per_million_tokens.items():
        cost_per_post = (avg_input_tokens / 1000000) * c["input"] + (avg_output_tokens / 1000000) * c["output"]
        print(f"{m}: ${cost_per_post:.5f} (or per million posts: ${cost_per_post*1000000:.2f})")
        model_total_cost[m] += cost_per_post

print("\n\nTotal cost (per million posts):")
for m, c in model_total_cost.items():
    print(f"{m}: ${c*1000000:.2f}")



entry
Average input tokens: 262.67
Average output tokens: 2.00
Total cost per post:
gpt-4.1-nano: $0.00005 (or per million posts: $54.13)
gpt-5-mini: $0.00007 (or per million posts: $69.67)

planner
Average input tokens: 512.67
Average output tokens: 690.00
Total cost per post:
gpt-4.1-nano: $0.00065 (or per million posts: $654.53)
gpt-5-mini: $0.00151 (or per million posts: $1508.17)

research
Average input tokens: 1246.33
Average output tokens: 109.33
Total cost per post:
gpt-4.1-nano: $0.00034 (or per million posts: $336.73)
gpt-5-mini: $0.00053 (or per million posts: $530.25)

analyst
Average input tokens: 4276.67
Average output tokens: 185.33
Total cost per post:
gpt-4.1-nano: $0.00100 (or per million posts: $1003.60)
gpt-5-mini: $0.00144 (or per million posts: $1439.83)


Total cost (per million posts):
gpt-4.1-nano: $2049.00
gpt-5-mini: $3547.92
